In [1]:
import numpy as np
import pandas as pd
import re
# from nltk.corpus import stopwords 
# from nltk.stem.porter import PorterStemmer
# from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
 import nltk
 
 nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vincentzhao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Data Processing

In [3]:
twitter_data = pd.read_csv('data/training.1600000.processed.noemoticon.csv', names=["target", "ids", "date", "flag", "user", "text"])

In [4]:
twitter_data.columns

Index(['target', 'ids', 'date', 'flag', 'user', 'text'], dtype='object')

In [5]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


convert positive and negative from 0,4 to -1,1

In [6]:
(twitter_data['target'] - 2)/2

0         -1.0
1         -1.0
2         -1.0
3         -1.0
4         -1.0
          ... 
1599995    1.0
1599996    1.0
1599997    1.0
1599998    1.0
1599999    1.0
Name: target, Length: 1600000, dtype: float64

vectorize text data

In [12]:
from sentence_transformers import SentenceTransformer
import torch

In [15]:
device = torch.cuda.current_device() if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    "paraphrase-MiniLM-L6-v2",
    device = device
)

In [17]:
embeddings = retriever.encode(twitter_data['text'])

embeddings

array([[ 0.06508211, -0.07679299, -0.12622495, ..., -0.09275538,
        -0.09516039, -0.1079419 ],
       [-0.10582985, -0.34505373,  0.00617036, ..., -0.09631609,
         0.11548704,  0.15210912],
       [ 0.62591106,  0.75567585, -0.04665479, ..., -0.16667397,
        -0.18563898, -0.0842871 ],
       ...,
       [ 0.19772325, -0.35297117,  0.45256093, ..., -0.02705677,
        -0.1769398 , -0.38491836],
       [ 0.23646286,  0.30513304,  0.3293145 , ..., -0.16758446,
        -0.09090563, -0.00253769],
       [-0.19584289,  0.5287847 , -0.03551808, ..., -0.3618492 ,
        -0.16284265,  0.29656956]], dtype=float32)

seperate into training and testing data

In [18]:
X_train, X_test, Y_train, Y_test = train_test_split(embeddings, twitter_data['target'], test_size=0.2, stratify=twitter_data['target'])

creating model

In [19]:
model = LogisticRegression(max_iter=1000)

In [20]:
model.fit(X_train, Y_train)

LogisticRegression(max_iter=1000)

model accuracy

In [21]:
X_train_prediction = model.predict(X_train)

training_data_accuracy = accuracy_score(Y_train, X_train_prediction)
print(training_data_accuracy)

0.7734515625


In [22]:
X_test_prediction = model.predict(X_test)

testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)
print(testing_data_accuracy)

0.774134375


saving model

In [23]:
import pickle

In [26]:
filename = '../sentiment_analysis_model_transformers.sav'

pickle.dump(model, open(filename, 'wb'))

In [29]:
pred = model.predict(embeddings[0])

print(pred, twitter_data['target'][0])

ValueError: Expected 2D array, got 1D array instead:
array=[ 0.06508211 -0.07679299 -0.12622495 -0.21194685 -0.37130913 -0.00826471
  0.3785915  -0.08375602 -0.46897793 -0.21640076 -0.06643903 -0.11941078
  0.15829617  0.47883844 -0.2279938  -0.17423813 -0.26982185 -0.13763326
 -0.23200516  0.36163703 -0.2000405   0.41419592  0.22448881 -0.02415787
 -0.24928416 -0.6047167   0.1545506   0.3345941  -0.47248006 -0.04750497
  0.30256927  0.17677073 -0.07618343 -0.47670278  0.3131007   0.06442707
 -0.29676744  0.04129129  0.26438686  0.18893059  0.15592638 -0.28681895
 -0.14491712  0.09865984 -0.29145616  0.17820393  0.15251888  0.01244185
  0.5235606  -0.30115196 -0.26732656  0.11977021  0.10050709 -0.744225
  0.10310429  0.12596667  0.15902075  0.5404102   0.01969431 -0.12694304
  0.38473293 -0.0016581  -0.131385    0.13452388  0.23858182 -0.12930803
 -0.44654918 -0.52816045 -0.4701198   0.08241788 -0.41215932 -0.11316337
 -0.01908605 -0.01629536 -0.06175913 -0.19537945  0.02687036  0.023